In [16]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
#import pyaerocom as pya 
import cartopy as ccrs

## Loading the data

In [17]:
# Observational record (CRU)
path_crutem = 'shared-ns1000k/inputs/CRU/CRUTEM.4.6.0.0.anomalies.nc'
path_hadcrut = 'shared-ns1000k/inputs/CRU/HadCRUT.4.6.0.0.median.nc'
  
crutem_data = xr.open_dataset(path_crutem)
hadcrut_data = xr.open_dataset(path_hadcrut)
# Historical model data  
    
# CESM2 
path_cesm2 = 'shared-cmip6-for-ns1000k/historical/CESM2/r1i1p1f1/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc'
tas_cesm2_data = xr.open_dataset(path_cesm2)

# NorEsm2-LM
 



/opt/conda/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


## Weigthed averages 

In [24]:
# Get the area weight 
path_area_weight_cesm2 = 'shared-cmip6-for-ns1000k/historical/CESM2/r1i1p1f1/areacella_fx_CESM2_historical_r1i1p1f1_gn.nc'
areacella_cesm2 = xr.open_dataset(path_area_weight_cesm2)

/opt/conda/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'areacella' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


In [25]:
# Function to calculate the weighted average 
def masked_average(xa, dim=None, weights=None, mask=None):
    """
    This function will average 
    :param xa: dataArray
    :param dim: dimension or list of dimensions. e.g. 'lat' or ['lat','lon','time']
    :param weights: weights (as xarray)
    :param mask: mask (as xarray), True where values to be masked.
    :return: masked average xarray
    """
    _ = xa.copy()
    if mask is not None:
        dum, mask_alld = xr.broadcast(xa, mask) # broadcast to all dims
        xa_ = xa_.where(np.logical_not(mask))
        if weights is not None:
            dum, weights_alld = xr.broadcast(xa, weights) # broadcast to all dims
            weights_alld = weights_alld.where(np.logical_not(mask_alld))
            return (xa_*weights_alld).sum(dim=dim)/weights_alld.sum(dim=dim)
        else:
            return xa_.mean(dim)
    elif weights is not None:
        dum, weights_alld = xr.broadcast(xa, weights) # broadcast to all dims
        return (xa_*weights_alld).sum(dim)/weights_alld.where(xa_.notnull()).sum(dim=dim)
    else:
        return xa.mean(dim)

<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: nbnd
Data variables:
    areacella  (lat, lon) float32 ...
    lat_bnds   (lat, nbnd) float32 ...
    lon_bnds   (lon, nbnd) float32 ...
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                15
    cesm_casename:          b.e21.BHIST.f09_g17.CMIP6-historical.001
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-16T21:43:39Z
    data_specs_version:     01.00.29
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    forcing_index:          1
    frequency:              fx
    grid:                   native 0.9x1.25 finite volume grid (192x288 latxlon)
    grid_label:           

In [ ]:
# Yearly average of the data 
#daily_t2m_1999 = t2m_1999.resample(time = 'D').mean()
tas_cesm_yearly = tas_cesm2_data['tas'].resample(time = '1Y').mean()

In [15]:
# Global average 


KeyError: 'tas'

In [9]:
tas_cesm_global

<xarray.DataArray 'tas' ()>
array(277.92297, dtype=float32)

In [8]:
# Time series of CESM data 


<xarray.DataArray 'tas' (year: 165, lat: 192, lon: 288)>
array([[[224.4655 , 224.46547, 224.46547, ..., 224.46547, 224.46547,
         224.46547],
        [224.89612, 224.86778, 224.72174, ..., 224.89752, 224.90123,
         224.89964],
        [225.38698, 225.36232, 225.35796, ..., 225.5744 , 225.53694,
         225.48076],
        ...,
        [256.6074 , 256.6331 , 256.657  , ..., 256.54022, 256.56216,
         256.58295],
        [256.29343, 256.30157, 256.30997, ..., 256.26462, 256.27466,
         256.28442],
        [255.96272, 255.96405, 255.96526, ..., 255.95795, 255.95967,
         255.96129]],

       [[225.02513, 225.02513, 225.02513, ..., 225.02513, 225.02513,
         225.02513],
        [225.47853, 225.44695, 225.2979 , ..., 225.48708, 225.48834,
         225.48439],
        [225.96367, 225.9362 , 225.92958, ..., 226.15318, 226.1164 ,
         226.05931],
        ...,
        [256.69946, 256.71533, 256.72977, ..., 256.6564 , 256.67044,
         256.68372],
        [256.53